# Writing elegant Python -- part 2

In [1]:
%run preamble.py

This chapter picks up from the earlier chapter "Writing elegant Python -- part 1". 


## Data Classes (if mutable)

Data Classes are a new feature of Python 3.7. They have been [backported](https://github.com/ericvsmith/dataclasses) to Python 3.6 as well, and can be installed with:

In [ ]:
pip install dataclasses

Data Classes are helper methods that will build a Python class and automatically generate the init function along with checks for different object types (along with default values) using Python's new type-hinting syntax. It is simple to make a dataclass simply by decorating a Python class:

In [48]:
from dataclasses import dataclass

In [49]:
@dataclass
class Currency:

    name: str
    central_bank: str
    symbol: str
    code: str = 'AUD'

    def get_exchange_rate(self, base='EUR'):
        url = 'http://data.fixer.io/api/latest'
        params = {
            'access_key': '93136301b1c8a659c34b8ce6bb63d0fa',
            'symbols': self.code,
            'base': base
        }
        response = requests.get(url, params)
        response.raise_for_status()
        return response.json()['rates'][self.code]

In [50]:
euro = Currency('Euro', 'European Central Bank', '€', 'EUR')
aus_dollar = Currency('Australian Dollar', 'Reserve Banks of Australia', '$')

In [51]:
euro

Currency(name='Euro', central_bank='European Central Bank', symbol='€', code='EUR')

In [52]:
aus_dollar

Currency(name='Australian Dollar', central_bank='Reserve Banks of Australia', symbol='$', code='AUD')

In [53]:
aus_dollar.get_exchange_rate(base=euro.code)

1.579552

The biggest advantage of the dataclass is that you can add new methods to the class, and modify the object after creation. Add to this that you don't have to spend time implementing the stock standard `__init__`, equality, and other methods and you have an extremely useful tool.

### Exercise: Updating the Currency dataclass

1. Make the `central_bank` optional (default value `None`).

2. Modify the dataclass above for currency to include an `amount` attribute which should be a `float`, defaulting to `1.0`.

3. Add a new method `convert_to(self, target_currency: Currency)` to the class that returns a new `Currency` object in the target currency with the correct value in the `amount` attribute.

For example: if you have SGD 100, how much is this worth in USD?

## Keep exception blocks small

Keep exception-handling blocks as small as possible, and catch specific
exceptions.

Exceptions in Python are not scary. You should think of them instead as extremely useful tools in finding out what has gone wrong in incoming data that you may have no control over. This is in contrast to using exception handling to gloss over issues with your code. Rather than using exception handling here you should be writing unit tests to test and fix your code! Part of using exceptions to your advantage is to write small, explicit exception handling blocks in your code rather than large broad exceptions which don't tell you what's wrong.

Consider this scenario: Every day you run a process where you read from an internal database containing a list of stock options to watch. Taking this information you must connect to an external API and retrieve the closing prices of these stocks over the last 12 months, along with the current price. You could write your code like this:

In [54]:
# What not to do
try:
    def stocks_of_interest():
        """
        Get the list of stocks from the database
        """
        #...
        pass

    def query_stock_prices(stock_symbol):
        """
        Get stock prices from external API
        """
        pass

    stocks = stocks_of_interest()
    for stock in stocks:
        data = query_stock_prices(stock)

    # and so on
except:
    pass

This code silently fails with no explanation why. There are two obvious sources of failure:

1. The connection to the database fails and for whatever reason doesn't return any data. This is a failure within your system somewhere.
2. The query of stock prices fails when you query the external API. This is a failure of a service you're paying money to be part of.

With the code written like this you don't know where the failure is. Rewriting the code to catch exceptions on discrete pieces of work gives you a much better idea of what's gone wrong:

In [55]:
def stocks_of_interest():
    """
    Get the list of stocks from the database
    """
    #...
    return ['AAPL', 'MSFT']

def query_stock_prices(stock_symbol):
    """
    Get stock prices from external API
    """
    raise ValueError("Could not connect to server")

try:
    stocks = stocks_of_interest()
except IOError as e:
    print(str(e))

for stock in stocks:
    try:
        data = query_stock_prices(stock)
    except ValueError as e:
        print(f"Could not retrieve value for stock {stock} - {e}")


Could not retrieve value for stock AAPL - Could not connect to server
Could not retrieve value for stock MSFT - Could not connect to server


You should also consider that you an use exception handling to handle different types of errors explicitly. For example you could have separate errors for failing to connect to the external stock price service at all vs not being able to find a given stock price symbol, and handle them differently:

In [56]:
class NoServerConnection(Exception):
    pass
class NoStockFound(Exception):
    pass

def query_stock_prices(stock_symbol):
    """
    Get stock prices from external API
    """
    if stock_symbol == 'AAPL':
        raise NoStockFound('Could not find stock in service')
    raise NoServerConnection("Could not connect to server")


for stock in stocks:
    try:
        data = query_stock_prices(stock)
    except NoStockFound as e:
        print(f"Could not retrieve value for stock {stock} - {e}")
    except NoServerConnection as e:
        print(f'Could not connect to server')
        raise  # deliberately throw an error if you can't connect to the external server

Could not retrieve value for stock AAPL - Could not find stock in service
Could not connect to server


NoServerConnection: Could not connect to server

### Exercise: debug this and rewrite the exception handling:

In [38]:
def get_hash(file):
    """
    Returns the MD5 hex digest for a file's contents
    """
    try:
        with open(file, mode='rb') as f:
            data = f.read()
        return hashlib.md5(data).hex_digest()
    except:
        return None

result = get_hash('/data/AAPL.h5')
print(result)

None


## Define `__iter__` on classes as a generator function

When you are writing your own `class` objects in Python you may be familiar with the `__next__` method which, when implemented in a class, allows you to define a loop over that class and return some values, raising a `StopIteration` exception when the list is exhausted. Consider the simple example of a counter:

In [82]:
@dataclass
class SimpleFibonacci:

    stop: int

    current = 1
    increment = 0

    def __iter__(self):
        return self

    def __next__(self):
        current = self.current
        self.current += self.increment
        self.increment = current

        if current > self.stop:
            raise StopIteration
        return current

In [83]:
list(SimpleFibonacci(20))

[1, 1, 2, 3, 5, 8, 13]

However, if you don't need to maintain the state of the iterator you can yield from the `__iter__` method instead. When there are no more yield statements the `StopIteration` exception will be raised automatically. The most basic example is:

In [100]:
class YieldZeroOne:

    def __iter__(self):
        yield 0
        yield 1

In [101]:
list(YieldZeroOne())

[0, 1]

### Exercise: Implement `__iter__` to return only active services

Given this code example write an `__iter__` method that loops over `self.services` and yields the service only if it is active.

In [99]:
# Example: define a class for a Server that allows iteration only through active services:
class Server:
    """Taken from  https://opensource.com/article/18/4/elegant-solutions-everyday-python-problems"""

    services = [
        {'active': False, 'protocol': 'ftp', 'port': 21},
        {'active': True, 'protocol': 'ssh', 'port': 22},
        {'active': True, 'protocol': 'http', 'port': 21},
    ]

    def __iter__(self):
        # Implement solution here
        pass

Test your solution with:

In [98]:
server = Server()
for service in server:
    print(service['protocol'], service['port'])

ssh 22
http 21


## Use `islice` to extract the first `n` items from a generator

Assume that you've written a generator that will keep generating new values indefinitely. Modifying the Fibonacci example from earlier you could have:

In [102]:
class FibonacciInfinite:

    def __iter__(self):
        current = 1
        increment = 0
        while True:
            yield current
            current, increment = current + increment, current

In [105]:
for i, f in enumerate(FibonacciInfinite()):
    if i >= 5:
        break
    print(f)

1
1
2
3
5


This can be a bit of a pain as you have to write loops or list comprehensions to capture these values. Instead, you can use the inbuilt `itertools` module and the `islice` function to extract values from the iterator:

In [106]:
from itertools import islice

In [110]:
islice(FibonacciInfinite(), 10)

This returns a generator of its own (in fact you are wrapping your infinite generator in another generator), but you can easily cast this as a `list` to extract the values:

In [109]:
list(islice(FibonacciInfinite(), 10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

As a bonus, you can add optionally use the start, stop, step syntax similar to the `range` function:

In [111]:
list(islice(FibonacciInfinite(), 0, 10, 2))

[1, 2, 5, 13, 34]

*Note* like `range` the step defaults to `1`, but unlike range the step must always be a positive integer.

## Use `itertools` for getting combinations of collections

The `itertools` module is actually extremely useful if you are dealing with many different iterable objects or collections of data. Consider you have some set of people you are trying to put into groups:

In [114]:
people = ['Henry', 'Hannah', 'Marcie']

You know the formula for permutations: $^nP_k = \frac{n!}{(n-k)!}$, and you could write a series of nested for loops to figure out groups of 2 like so (expecting 6 possible permutations with 3 people, given order is important):

In [115]:
for a in people:
    for b in (p for p in people if a != p):
        print(a, b)

Henry Hannah
Henry Marcie
Hannah Henry
Hannah Marcie
Marcie Henry
Marcie Hannah


Of course as the collection of people gets larger, and the number of people in the group get larger this becomes a deep nested for loop, and difficult to maintain. Out of the box itertools comes with a `permutations` function:

In [116]:
from itertools import permutations

In [117]:
for a, b in permutations(people, 2):
    print(a, b)

Henry Hannah
Henry Marcie
Hannah Henry
Hannah Marcie
Marcie Henry
Marcie Hannah


If you wish to calculate permutations with replacement you can use the `product` method. This method is a shortcut to writing nested for loops:

In [125]:
from itertools import product

In [126]:
for a, b in product(people, people):
    print(a, b)

Henry Henry
Henry Hannah
Henry Marcie
Hannah Henry
Hannah Hannah
Hannah Marcie
Marcie Henry
Marcie Hannah
Marcie Marcie


It also has a shortcut for repeating the same iterable object a number of times, yielding $n^k$ results:

In [127]:
for a, b in product(people, repeat=2):
    print(a, b)

Henry Henry
Henry Hannah
Henry Marcie
Hannah Henry
Hannah Hannah
Hannah Marcie
Marcie Henry
Marcie Hannah
Marcie Marcie


Likewise itertools comes with a `combinations` (where order is not important) method that returns $^nC_k=\frac{n!}{k!(n-k)!}$ results, and a `combinations_with_replacement` method that returns $(\binom{n}{k}) = \binom{n+k-1}{k} = \frac{(n + k - 1)!}{k!(n - 1)!}$ results:

In [118]:
from itertools import combinations, combinations_with_replacement

In [119]:
# expecting 3 results for combinations of size 2
for a, b in combinations(people, 2):
    print(a, b)

Henry Hannah
Henry Marcie
Hannah Marcie


In [123]:
# expecting 6 results for combinations with replacement with a group size of 2
for a in combinations_with_replacement(people, 2):
    print(*a)

Henry Henry
Henry Hannah
Henry Marcie
Hannah Hannah
Hannah Marcie
Marcie Marcie


### Exercise: Calculate the possible combinations for a bike lock

You have a bike lock where you can select the values $0 - 9$ on each of the $4$ wheels. Use the appropriate method from itertools to calculate all the possible combinations. What would be the effect of increasing the possible values on each wheel to 16?

In [6]:
# %load solutions/bike_lock.py

## Memoize the results of long running functions

Often a function will be completely deterministic based on the input, but takes a long time to run. For example, the following function returns the location of the closest city to a given latitude and longitude:

In [6]:
import requests
import time

In [7]:
def get_closest_city(latitude, longitude):
    time.sleep(5)
    url = 'https://llinvfu1jc.execute-api.ap-southeast-2.amazonaws.com/dev/location/nearest'
    params = {'latitude': latitude, 'longitude': longitude}
    response = requests.get(url, params)
    response.raise_for_status()
    return response.json()['name']

get_closest_city(52.4, 4.9)

'Amsterdam'

Given the location of the server, the length of time the server takes to execute commands (in this case we sleep for 5 seconds), and potential limitations on usage of the API, if you are going to keep calling this function it can have a dramatic effect on execution time.

Fortunately the `functools` library in Python contains a method for a `lru_cache` (LRU stands for Least Recently Used, which is the item that will be dropped from the cache when the cache size is exceeded). Using this as a Python decorator we can decorate the function call like so:

In [8]:
from functools import lru_cache

In [10]:
@lru_cache()
def get_closest_city(latitude, longitude):
    time.sleep(5)
    url = 'https://llinvfu1jc.execute-api.ap-southeast-2.amazonaws.com/dev/location/nearest'
    params = {'latitude': latitude, 'longitude': longitude}
    response = requests.get(url, params)
    response.raise_for_status()
    return response.json()['name']

get_closest_city(52.4, 4.9)

'Amsterdam'

The first time the call to the function will take the full amount of time. Subsequent calls to this function with the same arguments however will be much faster, simply retrieving the result from the cache:

In [12]:
%%timeit
get_closest_city(52.4, 4.9)

171 ns ± 2.35 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Implementing a persistent cache

It may be useful to implement a persistent cache in Python that stores results between sessions. The use case here is similar to memoization but has the extra step of saving the results to a persistent storage like disk involved as well. However to make this more generic we need to see some more complex processes:

First, if you want to decorate a function such that the results can be stored persistently for some given input arguments you need to be able to programatically get the function arguments, which can be done via the inbuilt `inspect` module:

In [14]:
def get_closest_city(latitude, longitude):
    time.sleep(5)
    url = 'https://llinvfu1jc.execute-api.ap-southeast-2.amazonaws.com/dev/location/nearest'
    params = {'latitude': latitude, 'longitude': longitude}
    response = requests.get(url, params)
    response.raise_for_status()
    return response.json()['name']

In [20]:
import inspect

In [23]:
sig = inspect.signature(get_closest_city)
sig.parameters.keys()

For ease of use you'll want to store strings of the incoming values. This does mean that for custom classes you should implement an appropriate string method. You'll use the `pickle` library to store the objects directly.

In [40]:
import pickle

In [46]:
pickle.dumps(43.5)

b'\x80\x03G@E\xc0\x00\x00\x00\x00\x00.'

In [47]:
pickle.loads(pickle.dumps(43.5))

43.5

To store the data you'll use the inbuilt `sqlite3` module to create a sqlite (on-disk) database for storage. In this case you'll use the `wraps` method in `functools` to build your decorator function:

In [57]:
import sqlite3
from functools import wraps
from itertools import chain

In [76]:
def persistent_cache_wrapper(database_location=':memory:'):
    def decorator(f):
        """Create the storage cache and return the decorated function"""

        sig = inspect.signature(get_closest_city)
        column_defs = ', '.join(
            f'{k} blob' for k in sig.parameters.keys()
        )
        conn = sqlite3.connect(database_location)
        create_table = f"""
        create table if not exists {f.__name__} ({column_defs}, _result blob)
        """
        c =  conn.cursor()
        c.execute(create_table)
        conn.commit()

        @wraps(f)
        def wrapper(*args, **kwargs):
            """Firstly query the decorated cache location"""
            ordered_incoming_args = chain(
                args,
                (kwargs.get(k) for k in
                 islice(sig.parameters.keys(), len(args), len(sig.parameters)))
            )
            pickled_args = [pickle.dumps(arg) for arg in ordered_incoming_args]
            where_statement = ' and '.join(f'{k} = ?' for k in sig.parameters.keys())
            sql_select = f"""select _result from {f.__name__} where {where_statement}"""
            c.execute(sql_select, pickled_args)
            result = c.fetchone()
            if result:
                print("Found result in Cache")
                return pickle.loads(result[0])

            # assuming no result is available, add it to the database

            result = f(*args, **kwargs)

            num_inserts = ','.join('?' * (len(sig.parameters) + 1))
            sql_insert = f"""insert into {f.__name__} values({num_inserts})"""
            c.execute(sql_insert, list(chain(pickled_args, [pickle.dumps(result)])))
            conn.commit()
            return result
        return wrapper
    return decorator

In [79]:
@persistent_cache_wrapper('solutions/closest_city.sqlite')
def get_closest_city(latitude, longitude):
    time.sleep(5)
    url = 'https://llinvfu1jc.execute-api.ap-southeast-2.amazonaws.com/dev/location/nearest'
    params = {'latitude': latitude, 'longitude': longitude}
    response = requests.get(url, params)
    response.raise_for_status()
    return response.json()['name']

In [80]:
get_closest_city(52.4, 4.9)

'Amsterdam'

In [81]:
get_closest_city(52.4, 4.9)

Found result in Cache


'Amsterdam'

In [83]:
get_closest_city(latitude=52.4, longitude=4.9)

Found result in Cache


'Amsterdam'

In [85]:
get_closest_city(-37.9, 145.0)

'Melbourne'